**Srinivas Naidu Pasyavula <br>
QC61851**
<br><br>
Dr. Tony Diana <br>
DATA 690 Introduction to NLP <br>
Homework, Week 7<br><br>
Exercise 4. Sentiment analysis with spaCy.

In [1]:
# This code mounts your Google Drive to the Colab notebook, changes the directory to a specific folder on your Drive, and lists the contents of that folder using the ls command.

#!hostname -I
from google.colab import drive
drive.mount('/content/drive')
#
%cd /content/drive/MyDrive/'Colab Notebooks'/'NLP DATA 690 UMBC'/'Week7'/'Exercise 4'
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/NLP DATA 690 UMBC/Week7/Exercise 4
 amazon_cells_labelled.txt					     Sentiment_Analysis_Dataset.csv
 imdb_labelled.txt						     yelp_labelled.txt
'QC61851-Srinivas Naidu Pasyavula-NLP-Week 7-Exercise 4- HW.ipynb'


In [2]:
# importing required libraries and packages
import pandas as pd

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [3]:
# Load datasets
data_amazon = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)
data_imdb = pd.read_csv('imdb_labelled.txt', sep='\t', header=None)
data_yelp = pd.read_csv('yelp_labelled.txt', sep='\t', header=None)

In [4]:
# Create combined dataframe
data_amazon.columns = ['Review', 'Label']
data_imdb.columns = ['Review', 'Label']
data_yelp.columns = ['Review', 'Label']

data_amazon['Company'] = 'Amazon'
data_imdb['Company'] = 'IMDb'
data_yelp['Company'] = 'Yelp'

combined_col = pd.concat([data_amazon, data_imdb, data_yelp], ignore_index=True)

In [5]:
# Explore the structure of the combined dataset
print(combined_col.head())

                                              Review  Label Company
0  So there is no way for me to plug it in here i...      0  Amazon
1                        Good case, Excellent value.      1  Amazon
2                             Great for the jawbone.      1  Amazon
3  Tied to charger for conversations lasting more...      0  Amazon
4                                  The mic is great.      1  Amazon


In [6]:
# infoof the data
print(combined_col.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Review   2748 non-null   object
 1   Label    2748 non-null   int64 
 2   Company  2748 non-null   object
dtypes: int64(1), object(2)
memory usage: 64.5+ KB
None


In [7]:
# Save the combined dataset to a CSV file
combined_col.to_csv('Sentiment_Analysis_Dataset.csv', index=False)

In [8]:
# Load the English language model in spaCy
nlp = spacy.load("en_core_web_sm")

In [9]:
# Class to clean the text
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [text_preprocessor(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

In [10]:
# function to clean the text
def text_preprocessor(text):
    # Lowercase the text
    text = text.lower()
    return text

In [11]:
# Tokenizing sentences
def spacy_tokenizer(sentence):
    tokens = nlp(sentence)
    tokens = [token.lemma_ for token in tokens if token.text.lower() not in STOP_WORDS and token.text not in string.punctuation]
    return tokens

In [12]:
# Vectorize and use LinearSVC as a classifier
pipe_countvect = Pipeline([
    ('cleaner', predictors()),
    # ('vectorizer', CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1, 1))),
    ('vectorizer', CountVectorizer(tokenizer=spacy_tokenizer, token_pattern=None, ngram_range=(1, 1))),
    ('classifier', LinearSVC())])

In [13]:
# Split the dataset into train and test sets
X = combined_col['Review']
y = combined_col['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Fit the data
pipe_countvect.fit(X_train, y_train)

# Predict with the test dataset
y_pred = pipe_countvect.predict(X_test)

In [15]:
# Print sample prediction results
for sample, pred in zip(X_test[:5], y_pred[:5]):
    print(f"Sample: {sample} Prediction: {pred}")

Sample: It's close to my house, it's low-key, non-fancy, affordable prices, good food. Prediction: 1
Sample: If you stay in Vegas you must get breakfast here at least once. Prediction: 1
Sample: Let's start with all the problemsthe acting, especially from the lead professor, was very, very bad.   Prediction: 0
Sample: It's too bad that everyone else involved didn't share Crowe's level of dedication to quality, for if they did, we'd have a far better film on our hands than this sub-par mess.   Prediction: 0
Sample: i felt insulted and disrespected, how could you talk and judge another human being like that? Prediction: 0


In [16]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8145454545454546
